# 

# Fine-Tuning Pre-Trained BERT Models Using the HuggingFace Library

## Summary
This notebook covers a simple use-case, where we'll be fine-tuning a BERT model using the Huggingface library.

In [1]:
#%%capture 
!pip install huggingface_hub transformers datasets evaluate
!pip install torch torchvision torchaudio
!pip install accelerate -U
!pip install scikit-learn
!pip install mlflow
!pip install ipywidgets
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import os
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
from huggingface_hub import notebook_login
import matplotlib.pyplot as plt
import evaluate
import mlflow

In [3]:
%matplotlib inline 

## 1. Load the dataset
We'll be using the Yelp review dataset and fine-tuning the BERT model on it. Let's load the dataset and view a single data point.

In [4]:
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

## 2. Process the dataset
Next, we need to tokenize the dataset. We'll be using the BERT cased tokenizer to do this.

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Since this notebook is simply designed to demonstrate how to fine-tune a cased BERT model, we'll select 1000 random examples for the fine-tuning and evaluation step.

In [7]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## 3. Load BERT
We'll load the BERT model from the Huggingface library. Note that this model has 5 labels.

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We'll also set up the training arguments. This model will be pushed to the hf-bert-finetuning repository on HF.

In [9]:
training_args = TrainingArguments(
    hub_model_id="hf-bert-finetuning",
    do_train=True,
    output_dir="bert-yelp",
    num_train_epochs=300,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
)

## Set up metrics that will be used during BERT fine-tuning
We'll treat this as a classification problem, and be tracing the accuracy of the model.

In [10]:
metric = evaluate.load("accuracy")

The following wrapper calculates the accuracy between the model output and the reference labels.

In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Set up BERT model training
Finally we'll set up a model Trainer object for BERT.

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

## Fine-tune the BERT model
We'll be tracing the results using the MLFlow library.

In [13]:
os.environ["MLFLOW_EXPERIMENT_NAME"] = "bert-yelp-hf-finetuning"
os.environ["MLFLOW_FLATTEN_PARAMS"] = "1"

trainer.train()
mlflow.end_run()

2024/04/28 16:38:26 INFO mlflow.tracking.fluent: Experiment with name 'bert-yelp-hf-finetuning' does not exist. Creating a new experiment.


  0%|          | 0/37500 [00:00<?, ?it/s]

{'loss': 0.9954, 'grad_norm': 10.145700454711914, 'learning_rate': 4.933333333333334e-05, 'epoch': 4.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.3348311185836792, 'eval_accuracy': 0.51, 'eval_runtime': 18.9848, 'eval_samples_per_second': 52.674, 'eval_steps_per_second': 6.584, 'epoch': 4.0}
{'loss': 0.3575, 'grad_norm': 40.292293548583984, 'learning_rate': 4.866666666666667e-05, 'epoch': 8.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.409299373626709, 'eval_accuracy': 0.488, 'eval_runtime': 17.871, 'eval_samples_per_second': 55.957, 'eval_steps_per_second': 6.995, 'epoch': 8.0}


Push the model to our HF space.

In [ ]:
trainer.push_to_hub() 

In [ ]:
 !ls -l mlruns/1

In [ ]:
run = "1/1a4ab6fd80784bd6a83b447fd01a45ca" 

In [ ]:
def plot_metric(metric_name):
    metric = pd.read_csv(
        f"mlruns/{run}/metrics/{metric_name}", sep=" ", names=["timestamp", "value", "steps"]
    )
    plt.plot(metric["steps"], metric["value"])

In [ ]:
plot_metric("loss") 

In [ ]:
plt.clf()
plot_metric("learning_rate")